### Training Process
This notebook will contain our main training and testing process for better modulisation and visualisation

In [1]:
import torch
import torch.nn as nn
from pipeline import RobertaGCN, TextClassificationDataset
import tqdm
from transformers.models.roberta import RobertaTokenizer
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader
from utils import read_train_data, read_train_split, read_test_data

PARAMS = {
    "lr": 2e-5,
    "batch_size": 64
}

# Training function
def train_model(model, train_loader, val_loader, num_epochs=5):
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)
    criterion = nn.CrossEntropyLoss()
    
    best_val_f1 = 0.0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            input_ids = batch['input_ids'].to(model.device)
            attention_mask = batch['attention_mask'].to(model.device)
            labels = batch['label'].to(model.device)
            
            optimizer.zero_grad()
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                input_ids = batch['input_ids'].to(model.device)
                attention_mask = batch['attention_mask'].to(model.device)
                labels = batch['label'].to(model.device)
                
                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_val_loss = val_loss / len(val_loader)
        val_acc = accuracy_score(all_labels, all_preds)
        val_f1 = f1_score(all_labels, all_preds, average='binary' if len(set(all_labels)) == 2 else 'macro')
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {avg_train_loss:.4f}")
        print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")
        
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            # Save the model
            torch.save(model.state_dict(), 'best_roberta_gcn_model.pth')
            print("Model saved!")
        
        print("-" * 50)
    
    return model

In [ ]:
# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Load your data
# This is a placeholder - replace with your actual data loading code

train_data, dev_data = read_train_split()
texts_train = train_data['text']
labels_train = train_data['label']
texts_val = dev_data['text']
labels_val = dev_data['label']

# Create datasets
train_dataset = TextClassificationDataset(texts_train, labels_train, tokenizer)
val_dataset = TextClassificationDataset(texts_val, labels_val, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=PARAMS['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=PARAMS['batch_size'])

# Initialize model
model = RobertaGCN(num_classes=5)

# Train model
trained_model = train_model(model, train_loader, val_loader)

print("Training completed!")

loading file vocab.json from cache at C:\Users\Xinyang\.cache\huggingface\hub\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd80ffbac7b\vocab.json
loading file merges.txt from cache at C:\Users\Xinyang\.cache\huggingface\hub\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd80ffbac7b\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\Xinyang\.cache\huggingface\hub\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd80ffbac7b\tokenizer_config.json
loading file tokenizer.json from cache at C:\Users\Xinyang\.cache\huggingface\hub\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd80ffbac7b\tokenizer.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at C:\Users\Xinyang\.cache\huggingface\hub\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd80ffbac7b\con

Series([], Name: text, dtype: object)


ValueError: num_samples should be a positive integer value, but got num_samples=0